# VLF analysis
<hr style="border-top-width: 4px; border-top-color: #34609b;">

First import the ROOT Python module.

In [ ]:
from Generic import *

In [ ]:
ROOT.gROOT.ProcessLine("#include <math.h>");
ROOT.gSystem.Load("libDelphes");
ROOT.gROOT.Macro("rootphi.C");

Optional: activate the JavaScript visualisation to produce interactive plots.

In [ ]:
%jsroot on

Defining function to open sets of files in a single chain

In [ ]:
#The signal
DTrees={};
MainChain=ROOT.TChain("Delphes");
MainChain.Add("/eos/user/j/jruizalv/VLF_Samples/SSSFDM_150Deltam10.root")
DTrees['Signal']=MainChain;

In [ ]:
DTrees

In [ ]:
#The backgrounds
Path1= "/eos/cms/store/user/nvanegas/"; 
#Path1= "/eos/user/n/nvanegas/VLF_analysis/"
ListOfFolders = ["BckgndSingleTop","BckgndW+Jets","BckgndWW","BckgndWZ"];
Path2= "tag_1_delphes_events*";

In [ ]:
for Folder in ListOfFolders:
    Path="root://eoscms.cern.ch/"+Path1+Folder+"/*.root"
    MainChain=ROOT.TChain("Delphes");
    MainChain.Add(Path)
    DTrees[Folder]=MainChain;

In [ ]:
Trigger="(MissingET.MET>60.)&&(Jet[0].PT>60.)&&(Muon[0].PT>5.)"
Cut0="(Muon_size==1)"

FullCut=Trigger+"&&"+Cut0

TIMEStr=time.strftime("%H_%M_%S_%d_%m_%y")
PDFNames="Plots_Wjets_Signal_150DM10_"+TIMEStr
print "Plotting on:", PDFNames
HistFile= ROOT.TFile(PDFNames+".root", "recreate")

HToExtract=["Jet_size","Jet[0].PT","Jet[1].PT","Muon_size","Muon[0].PT","MissingET.MET",
            "TMath::Sqrt(2*Muon[0].PT*MissingET.MET*(1-TMath::Cos(deltaPhi(MissingET.Phi,Muon[0].Phi))))",#WE NEED TO INCLUDE MT DEFINITION
            "TMath::Abs(Jet[0].PT-MissingET.MET)/(Jet[0].PT+MissingET.MET)",
            "TMath::Abs(ScalarHT.HT-MissingET.MET)/(ScalarHT.HT+MissingET.MET)"]

Binning=["(15,0,15)","(100,0,1000)","(50,0,500)","(5,0,5)","(50,0,100)","(100,0,1000)",
         "(100,0,500)","(200,0,2)","(200,0,2)"]

EntriesListWjets=[]
EntriesListSignal=[]
c = ROOT.TCanvas()
for i in xrange(len(HToExtract)):
    HistName=HToExtract[i].replace("(","").replace(")","").replace("[","").replace("]","").replace(":","").replace(".","").replace("-","m").replace("+","p").replace(" ","_").replace("&&","_and_").replace("&","_and_").replace("*","_").replace(">","g").replace("<","l").replace(",","l").replace("/","d")
    print "Extracting "+HToExtract[i]
    #Extracting Wjets histogram
    HistNameWjets=HistName+"_Wjets"
    DTrees["BckgndW+Jets"].Draw(HToExtract[i]+" >> "+HistNameWjets+Binning[i],FullCut)
    TemHisto1=ROOT.gDirectory.Get(HistNameWjets)
    EntriesListWjets.append(TemHisto1.GetEntries())
    TemHisto1.Sumw2()
    TemHisto1.Scale(1./TemHisto1.Integral())
    TemHisto1.Write()
    TemHisto1.Draw()
    #del(TemHisto1)
    #Extracting Signal histogram
    HistNameSig=HistName+"_Sig"
    DTrees["Signal"].Draw(HToExtract[i]+" >> "+HistNameSig+Binning[i],FullCut)
    TemHisto2=ROOT.gDirectory.Get(HistNameSig)
    EntriesListSignal.append(TemHisto2.GetEntries())
    TemHisto2.Sumw2()
    TemHisto2.Scale(1./TemHisto2.Integral())
    TemHisto2.Write()
    TemHisto2.Draw("same")
    c.Draw()
    print "Wjets entries=", EntriesListWjets[-1]
    print "Signal entries=", EntriesListSignal[-1]
    print "-----------------------------------------"
    #del(TemHisto2)

print "Wjets cut flow:", EntriesListWjets
print "Signal cut flow:", EntriesListSignal

HistFile.Close()

Cuts optimization

In [ ]:
VariablesToCut=[['l',"","Muon[0].PT","(500,0,500)",FullCut],
                ['l',"","TMath::Sqrt(2*Muon[0].PT*MissingET.MET*(1-TMath::Cos(deltaPhi(MissingET.Phi,Muon[0].Phi))))","(500,0,500)"
                 ,FullCut+"&&(Muon[0].PT<20)"],
                ['l',"","TMath::Abs(ScalarHT.HT-MissingET.MET)/(ScalarHT.HT+MissingET.MET)","(200,0,2)"
                 ,FullCut+"&&(Muon[0].PT<20)"+"&&(TMath::Sqrt(2*Muon[0].PT*MissingET.MET*(1-TMath::Cos(deltaPhi(MissingET.Phi,Muon[0].Phi))))<50)"]
               ]

XSWjets=3091.5; # pb
XSSig=0.100 #0.008599; # pb
lumi=100 # fb-1
ws=(lumi*XSSig*1000.)/SigChain.GetEntries()
wb=(lumi*XSWjets*1000.)/WjetsChain.GetEntries()
print "Signal weight =", ws
print "Signal events =", SigChain.GetEntries()
print "Wjets weight = ", wb
print "Wjets events = ", WjetsChain.GetEntries()

In [ ]:
TIMEStr=time.strftime("%H_%M_%S_%d_%m_%y")
PDFNames="Variables_Efficiencies_WJets_DeltamM10_"+TIMEStr
print "Plotting on:", PDFNames

XVariableAxis=["MET",
               "p_{T}(j_{1})",
               "p_{T}(j_{2})",
               "M_{T}(MET,#mu)",]
XVariableName=["MET",
               "PT1",
               "PT2",
               "MT MET MU"]
EffFile= ROOT.TFile(PDFNames+".root", "recreate")

for i in xrange(len(VariablesToCut)):
    CurCanvScan = rootnotes.canvas("MyPlot", (600, 600))

    #Constructing efficiency histograms
    Var=i
    SignalMETEffHisto = GetEffHisto(VariablesToCut[Var][0],
                                VariablesToCut[Var][1],
                                SigChain,
                                "Signal",
                                VariablesToCut[Var][2],
                                VariablesToCut[Var][3],
                                VariablesToCut[Var][4])

    WjetsMETEffHisto = GetEffHisto(VariablesToCut[Var][0],
                                VariablesToCut[Var][1],
                                WjetsChain,
                                "Wjets",
                                VariablesToCut[Var][2],
                                VariablesToCut[Var][3],
                                VariablesToCut[Var][4])

    Z1Histo,ZAsimovHisto, SHisto, BHisto, Z3Histo, Z4Histo = GetZHistos(VariablesToCut[Var][0],
                                VariablesToCut[Var][1],
                                SigChain, ws,
                                WjetsChain, wb,
                                VariablesToCut[Var][2],
                                VariablesToCut[Var][3],
                                VariablesToCut[Var][4])

    METSoverWjets=SignalMETEffHisto.Clone("METSoverWjets")
    METSoverWjets.Sumw2(); METSoverWjets.Divide(WjetsMETEffHisto)
    
    #Histogram cosmetics
    WjetsMETEffHisto.SetStats(ROOT.kFALSE)
    WjetsMETEffHisto.SetLineStyle(7); SignalMETEffHisto.SetLineStyle(9); METSoverWjets.SetLineStyle(8)
    Z1Histo.SetLineStyle(6)#; ZAsimovHisto.SetLineStyle(2)
    WjetsMETEffHisto.SetLineWidth(2); SignalMETEffHisto.SetLineWidth(1); METSoverWjets.SetLineWidth(2)
    #Z1Histo.SetLineWidth(3); ZAsimovHisto.SetLineWidth(3)
    WjetsMETEffHisto.SetLineColor(ROOT.kRed); SignalMETEffHisto.SetLineColor(ROOT.kGreen+1); METSoverWjets.SetLineColor(ROOT.kBlack)
    Z1Histo.SetLineColor(ROOT.kBlue)#; ZAsimovHisto.SetLineColor(ROOT.kOrange)
    #Z1Histo.SetFillColor(ROOT.kBlue); ZAsimovHisto.SetFillColor(ROOT.kOrange)
    #Z1Histo.SetFillStyle(3004); ZAsimovHisto.SetFillStyle(3005)
    ############################
    SHisto.SetStats(ROOT.kFALSE)
    SHisto.SetLineStyle(1); BHisto.SetLineStyle(2)
    SHisto.SetLineWidth(2); BHisto.SetLineWidth(1)
    SHisto.SetLineColor(ROOT.kGreen); BHisto.SetLineColor(ROOT.kRed)
    
    #Setting legend
    LEG=ROOT.TLegend(0.6,0.6,0.9,0.9)
    LEG.AddEntry(WjetsMETEffHisto, "#varepsilon(W+jets)", "l")
    LEG.AddEntry(ROOT.TObject(), GetMR(WjetsMETEffHisto), "")
    LEG.AddEntry(ROOT.TObject(), GetEWI(WjetsMETEffHisto), "")
    LEG.AddEntry(SignalMETEffHisto, "#varepsilon(S)", "l")
    LEG.AddEntry(ROOT.TObject(), GetMR(SignalMETEffHisto), "")
    LEG.AddEntry(ROOT.TObject(), GetEWI(SignalMETEffHisto), "")
    LEG.AddEntry(METSoverWjets, "#varepsilon(S)/#varepsilon(W+jets)", "l")
    #LEG.AddEntry(ROOT.TObject(), GetMR(METSoverZvv), "")
    #LEG.AddEntry(ROOT.TObject(), GetEWI(METSoverZvv), "")
    LEG.AddEntry(Z1Histo, "s/#sqrt{b}", "l")
    #LEG.AddEntry(ZAsimovHisto, "Asimov", "l")
    LEG.SetFillColor(0)
    
    #Setting legend for signal and background histos
    LEGSB=ROOT.TLegend(0.6,0.6,0.9,0.9)
    LEGSB.AddEntry(SHisto, "S", "l")
    LEGSB.AddEntry(ROOT.TObject(), GetMR(SHisto), "")
    LEGSB.AddEntry(ROOT.TObject(), GetEWI(SHisto), "")
    LEGSB.AddEntry(BHisto, "Wjets", "l")
    LEGSB.AddEntry(ROOT.TObject(), GetMR(BHisto), "")
    LEGSB.AddEntry(ROOT.TObject(), GetEWI(BHisto), "")
    LEGSB.SetFillColor(0)
    
    #Setting maximal value of first plotted histogram
    Maxima=[WjetsMETEffHisto.GetMaximum(), SignalMETEffHisto.GetMaximum(), METSoverWjets.GetMaximum(),
            Z1Histo.GetMaximum()]#, ZAsimovHisto.GetMaximum()]
    WjetsMETEffHisto.SetMaximum(1.05*max(Maxima))
    
    #Setting maximal value of first plotted histogram
    MaximaSB=[SHisto.GetMaximum(), BHisto.GetMaximum()]
    SHisto.SetMaximum(1.05*max(MaximaSB))
    
    #Setting axis titles
    MinX=float(VariablesToCut[Var][3].split(",")[1])
    MaxX=float(VariablesToCut[Var][3].split(",")[-1][0:-1])
    Nbins=int(VariablesToCut[Var][3].split(",")[0][1:])
    WjetsMETEffHisto.SetTitle(";"+XVariableAxis[i]+";A.U./{0:.2f} ".format((MaxX-MinX)/Nbins)+"GeV")
    WjetsMETEffHisto.GetYaxis().SetTitleOffset(1.4)
    
    #Setting axis titles
    SHisto.SetTitle(";"+XVariableAxis[i]+";A.U./{0:.2f} ".format((MaxX-MinX)/Nbins)+"GeV")
    SHisto.GetYaxis().SetTitleOffset(1.4)
    
    #.GetXaxis().SetRangeUser(0,20)
    #.GetYaxis().SetRangeUser(0.1,6)
    #ROOT.gPad.SetLogy()
    
    #Plotting
    WjetsMETEffHisto.Draw("hist e"); SignalMETEffHisto.Draw("hist e same");
    METSoverWjets.Draw("E0 same")
    Z1Histo.Draw("E1 same")
    #ZAsimovHisto.Draw("E1 same")
    LEG.Draw()
    
    #if i==0:
    #    CurCanvScan.Print(PDFNames+".pdf(","Title:"+XVariableName[i])        
    ###elif i==len(VariablesToCut)-1:
    ###    CurCanvScan.Print(PDFNames+".pdf)","Title:"+XVariableName[i])
    #else:
    #    CurCanvScan.Print(PDFNames+".pdf","Title:"+XVariableName[i])
    #    
    ##Plotting
    #SHisto.Draw("hist e"); BHisto.Draw("hist e same")
    #LEGSB.Draw()
    #
    #if i==len(VariablesToCut)-1:
    #    CurCanvScan.Print(PDFNames+".pdf)","Title:"+XVariableName[i])
    #else:
    #    CurCanvScan.Print(PDFNames+".pdf","Title:"+XVariableName[i])
    
    WjetsMETEffHisto.Write(); SignalMETEffHisto.Write(); METSoverWjets.Write(); Z1Histo.Write()#; ZAsimovHisto.Write()
    SHisto.Write(); BHisto.Write(); Z3Histo.Write(); Z4Histo.Write()

EffFile.Close()

In [ ]:
h = ROOT.TH1F("TracksPt","Tracks;Pt [GeV/c];#",128,0,64)
for event in f.events:
    for track in event.tracks:
        h.Fill(track.Pt())
c = ROOT.TCanvas()
h.Draw()
c.Draw()